<a href="https://colab.research.google.com/github/itrinia/code-TA/blob/main/ta_ileene_v3_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# prompt: give me code to make sure the folders are filled and give the variables.
# /content/drive/MyDrive/TA Ileene/augmented/images
# /content/drive/MyDrive/TA Ileene/augmented/labels
# /content/drive/MyDrive/TA Ileene/val/images
# /content/drive/MyDrive/TA Ileene/val/labels

from google.colab import drive
drive.mount('/content/drive')

import os

def ensure_folders_exist(folder_paths):
  """Ensures that the specified folders exist.

  Args:
    folder_paths: A list of folder paths to create.
  """
  for path in folder_paths:
    if not os.path.exists(path):
      os.makedirs(path)
      print(f"Created folder: {path}")
    else:
      print(f"Folder already exists: {path}")

# Define the folder paths as variables.
augmented_images_path = "/content/drive/MyDrive/TA Ileene/augmented/images"
augmented_labels_path = "/content/drive/MyDrive/TA Ileene/augmented/labels"
val_images_path = "/content/drive/MyDrive/TA Ileene/val/images"
val_labels_path = "/content/drive/MyDrive/TA Ileene/val/labels"


# Create a list of the folder paths.
folder_paths = [
    augmented_images_path,
    augmented_labels_path,
    val_images_path,
    val_labels_path
]

# Call the function to ensure the folders exist.
ensure_folders_exist(folder_paths)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Folder already exists: /content/drive/MyDrive/TA Ileene/augmented/images
Folder already exists: /content/drive/MyDrive/TA Ileene/augmented/labels
Folder already exists: /content/drive/MyDrive/TA Ileene/val/images
Folder already exists: /content/drive/MyDrive/TA Ileene/val/labels


In [8]:
# Cell 2: Sanity‐check file counts
import os
print("→ Train images:", len(os.listdir(augmented_images_path)))
print("→ Train labels:", len(os.listdir(augmented_labels_path)))
print("→ Val   images:", len(os.listdir(val_images_path)))
print("→ Val   labels:", len(os.listdir(val_labels_path)))

→ Train images: 10550
→ Train labels: 10550
→ Val   images: 2262
→ Val   labels: 2262


# define model

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import (
    Conv2D, Activation, Add, MaxPooling2D,
    Conv2DTranspose, concatenate, Input, Multiply
)
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Sequence

In [12]:
# Define the residual block
def residual_block(x, filters):
    # First convolution layer
    conv1 = Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
    conv1 = Activation('relu')(conv1)

    # Second convolution layer
    conv2 = Conv2D(filters, kernel_size=(3, 3), padding='same')(conv1)
    conv2 = Activation('relu')(conv2)

    # Skip connection
    skip = Add()([x, conv2])
    return skip

# Define the attention mechanism
def attention_block(x, g, filters):
    # Query and Key
    theta_x = Conv2D(filters, kernel_size=(1, 1), strides=(1, 1), padding='same')(x)
    phi_g = Conv2D(filters, kernel_size=(1, 1), strides=(1, 1), padding='same')(g)

    # Add and apply ReLU
    add_xg = Add()([theta_x, phi_g])
    add_xg = Activation('relu')(add_xg)

    # Attention map
    psi = Conv2D(1, kernel_size=(1, 1), strides=(1, 1), padding='same')(add_xg)
    psi = Activation('sigmoid')(psi)

    # Apply attention
    return Multiply()([x, psi])


# Define the DocUNet model
def build_docunet(input_shape=(512, 512, 1)):
    inputs = Input(input_shape)

    # Encoder
    x = Conv2D(64, kernel_size=(3, 3), padding='same')(inputs)
    x = Activation('relu')(x)
    res1 = residual_block(x, 64)
    x = MaxPooling2D(pool_size=(2, 2))(res1)

    x = Conv2D(128, kernel_size=(3, 3), padding='same')(x)
    x = Activation('relu')(x)
    res2 = residual_block(x, 128)
    x = MaxPooling2D(pool_size=(2, 2))(res2)

    x = Conv2D(256, kernel_size=(3, 3), padding='same')(x)
    x = Activation('relu')(x)
    res3 = residual_block(x, 256)
    x = MaxPooling2D(pool_size=(2, 2))(res3)

    x = Conv2D(512, kernel_size=(3, 3), padding='same')(x)
    x = Activation('relu')(x)
    res4 = residual_block(x, 512)
    x = MaxPooling2D(pool_size=(2, 2))(res4)

    # Bottleneck with attention
    g = Conv2D(512, kernel_size=(3, 3), padding='same')(x)
    g = Activation('relu')(g)
    g = attention_block(g, x, 512)

    # Decoder
    x = Conv2DTranspose(512, kernel_size=(3, 3), strides=(2, 2), padding='same')(g)
    x = concatenate([x, res4])
    x = Activation('relu')(x)

    x = Conv2DTranspose(256, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = concatenate([x, res3])
    x = Activation('relu')(x)

    x = Conv2DTranspose(128, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = concatenate([x, res2])
    x = Activation('relu')(x)

    x = Conv2DTranspose(64, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = concatenate([x, res1])
    x = Activation('relu')(x)


    # Output layer
    outputs = Conv2D(1, kernel_size=(1, 1), activation='sigmoid')(x)

    # Build the model
    model = Model(inputs, outputs)
    return model

# Compile the model
model = build_docunet()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 512, 512,  │        640 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 512, 512,  │          0 │ conv2d[0][0]      │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 512, 512,  │     36,928 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 512, 512,  │          0 │ conv2d_1[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 512, 512,  │     36,928 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 512, 512,  │          0 │ conv2d_2[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 512, 512,  │          0 │ activation[0][0], │
│                     │ 64)               │            │ activation_2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 256, 256,  │          0 │ add[0][0]         │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 256, 256,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 256, 256,  │          0 │ conv2d_3[0][0]    │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 256, 256,  │    147,584 │ activation_3[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 256, 256,  │          0 │ conv2d_4[0][0]    │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 256, 256,  │    147,584 │ activation_4[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_5        │ (None, 256, 256,  │          0 │ conv2d_5[0][0]    │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 256, 256,  │          0 │ activation_3[0][… │
│                     │ 128)              │            │ activation_5[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 128, 128,  │          0 │ add_1[0][0]     

 Total params: 16,161,218 (61.65 MB)

 Trainable params: 16,161,218 (61.65 MB)

 Non-trainable params: 0 (0.00 B)

# Data Preparation
define a simple sequence using ImageDataGenerator or custom data loaders if my dataset isn't structured in a way that fits ImageDataGenerator directly. Since im working with paired images and labels (masks), i'll likely need a custom data loader.


In [13]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import Sequence

class ImageMaskGenerator(Sequence):
    def __init__(self, images_dir, masks_dir, batch_size, image_size, shuffle=True):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle

        # Load all image and mask file names
        self.image_filenames = os.listdir(images_dir)
        self.mask_filenames = os.listdir(masks_dir)

        # Ensure images and masks are in the same order
        self.image_filenames.sort()
        self.mask_filenames.sort()

    def __len__(self):
        return int(np.floor(len(self.image_filenames) / self.batch_size))

    def __getitem__(self, index):
        # Select batch
        batch_image_filenames = self.image_filenames[index * self.batch_size:(index + 1) * self.batch_size]
        batch_mask_filenames = self.mask_filenames[index * self.batch_size:(index + 1) * self.batch_size]

        images = []
        masks = []

        for img_name, mask_name in zip(batch_image_filenames, batch_mask_filenames):
            # Load image and mask, resize and normalize
            img_path = os.path.join(self.images_dir, img_name)
            mask_path = os.path.join(self.masks_dir, mask_name)

            img = load_img(img_path, target_size=self.image_size, color_mode="grayscale")
            mask = load_img(mask_path, target_size=self.image_size, color_mode="grayscale")

            img = img_to_array(img) / 255.0  # Normalize image
            mask = img_to_array(mask) / 255.0  # Normalize mask

            images.append(img)
            masks.append(mask)

        return np.array(images), np.array(masks)

    def on_epoch_end(self):
        if self.shuffle:
            # Shuffle data after every epoch
            temp = list(zip(self.image_filenames, self.mask_filenames))
            np.random.shuffle(temp)
            self.image_filenames, self.mask_filenames = zip(*temp)


# Create data generator

In [14]:
train_generator = ImageMaskGenerator(
    images_dir=augmented_images_path,
    masks_dir=augmented_labels_path,
    batch_size=16,
    image_size=(512, 512)
)

val_generator = ImageMaskGenerator(
    images_dir=val_images_path,
    masks_dir=val_labels_path,
    batch_size=16,
    image_size=(512, 512)
)

# Set Up Callbacks

In [15]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

checkpoint = ModelCheckpoint(
    'docunet_best_model.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

# Training model

In [16]:
model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[early_stopping, checkpoint],
    batch_size=16
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7d8384f0c220>

# Save model weights

In [ ]:
model.save_weights('docunet_final_weights.h5')